In [1]:
import numpy as np
from scipy.integrate import odeint

In [2]:
# Forward ODE system
def system(y, t, matrix):
    row_sums = np.sum(matrix, axis=1)
    shapes = 1 / (1 + np.exp(row_sums))
    dydt = shapes - y
    return dydt

# Jacobian of the system w.r.t. y (df/dy)
def jacobian(y, t, matrix):
    # df[i]/dy[j] = -1 if i == j, 0 otherwise (since dydt[i] = shapes[i] - y[i])
    return -np.eye(len(y))

# Adjoint ODE system (dλ/dt = -λ * df/dy)
def adjoint_system(lmbda, t, y_trajectory, t_forward, matrix):
    # Interpolate y at current t (since adjoint runs backward)
    y = np.interp(t, t_forward, y_trajectory[:, 0]), np.interp(t, t_forward, y_trajectory[:, 1]), np.interp(t, t_forward, y_trajectory[:, 2])
    y = np.array(y)
    J = jacobian(y, t, matrix)
    dlmbda_dt = -np.dot(lmbda, J)  # dλ/dt = -λ * df/dy
    return dlmbda_dt

# Gradient of f w.r.t. matrix (df/d(matrix))
def gradient_f_wrt_matrix(y, t, matrix):
    row_sums = np.sum(matrix, axis=1)
    z = 1 / (1 + np.exp(row_sums))
    dz_dsum = z * (1 - z)  # Sigmoid derivative
    grad = np.zeros_like(matrix)
    for i in range(matrix.shape[0]):
        grad[i, :] = dz_dsum[i]  # Each element in row i contributes equally to sum
    return grad

In [3]:
# Setup
np.random.seed(0)
matrix = np.random.normal(size=(3, 3))

n = matrix.shape[0]
y0 = np.array([0.5, 0.7, 0.3])
stop_time = 3
t = np.linspace(0, stop_time, 101)
target_values = np.array([0.5, 0.7, 0.3])

learning_rate = 1
max_iterations = 200
mse_target = 0.001

# Optimization loop
for iteration in range(max_iterations):
    # 1. Solve forward ODE
    solution = odeint(system, y0, t, args=(matrix,))
    solution_at_t1 = solution[-1]
    mse = np.mean((solution_at_t1 - target_values)**2)

    if iteration % 20 == 0:
        print(f"Solution at t = 1: {solution_at_t1}")
        print(f"Iteration {iteration}: MSE = {mse:.6f}")
        if mse < mse_target:
            break

    # 2. Solve adjoint ODE backward
    lambda0 = solution_at_t1 - target_values  # Initial condition: dL/dy(1)
    t_backward = t[::-1]  # Reverse time from 1 to 0
    adjoint_solution = odeint(adjoint_system, lambda0, t_backward, args=(solution, t, matrix))

    # Reverse adjoint solution to match forward time
    lambda_t = adjoint_solution[::-1]

    # 3. Compute gradient w.r.t. matrix
    gradient_matrix = np.zeros_like(matrix)
    dt = t[stop_time] - t[0]
    for k in range(len(t)):
        grad_f = gradient_f_wrt_matrix(solution[k], t[k], matrix)
        for i in range(n):
            gradient_matrix[i, :] += lambda_t[k][i] * grad_f[i, :] * dt  # Integrate λ * df/d(matrix)

    # Update matrix
    matrix += learning_rate * gradient_matrix

# Final results
solution_final = odeint(system, y0, t, args=(matrix,))
final_at_t1 = solution_final[-1]

print("\nFinal Results:")
print(f"Optimized matrix:\n{matrix}")
print(f"Solution at t = 1: {final_at_t1}")
print(f"Target values: {target_values}")
print(f"Final MSE: {np.mean((final_at_t1 - target_values)**2):.6f}")

Solution at t = 1: [0.0642039  0.07460741 0.33117453]
Iteration 0: MSE = 0.194002
Solution at t = 1: [0.49992984 0.69972176 0.30000307]
Iteration 20: MSE = 0.000000

Final Results:
Optimized matrix:
[[ 0.71650161 -0.64739352 -0.06881275]
 [ 0.9152008   0.5418656  -2.30297027]
 [ 1.00067846 -0.10076717 -0.05262881]]
Solution at t = 1: [0.49992984 0.69972176 0.30000307]
Target values: [0.5 0.7 0.3]
Final MSE: 0.000000
